# Installing Required Libraries

In [ ]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=05e1a826bd2638b169a554be51fd88482d1a544612fa67bccb4ed24aa48f2ecc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef179

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
! pip install opendatasets

In [ ]:
import opendatasets as od
import pandas as pd
import numpy as np
import time
import gc
import json

import nltk
import evaluate
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import math
import torch.nn as nn

# Data

Kaggle username: sallyzhao<br>
Key: 1d245367f7ccd8d3e6c7f97663c2af02

In [ ]:
#Download the dataset (use the above username and key when prompted)
od.download("https://www.kaggle.com/datasets/konradb/chain-of-thought-collection/data/CoT_collection.json")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sallyzhao
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/konradb/chain-of-thought-collection


100%|██████████| 1.17G/1.17G [00:15<00:00, 83.2MB/s]


In [ ]:
#Specify file path that will be used to extract the actual data below
file_path = "./chain-of-thought-collection/flan_collection_subset.json"

In [ ]:
# The data needs to be processed in chunks to avoid a long runtime, and consequently a runtime disconnect
# Initialize an empty list to store processed chunks
chunks = []
chunk_size = 1000  # Number of items to process per chunk

# Open the JSON file and load it line by line
with open(file_path, 'r') as file:
    data = json.load(file)  # Load the entire file as a dictionary

    # Convert the dictionary to an iterable list of items (key-value pairs)
    items = list(data.items())

    # Process in chunks
    for i in range(0, len(items), chunk_size):
        chunk = items[i:i+chunk_size]

        # Convert the chunk into a DataFrame
        chunk_df = pd.DataFrame([{"id": key, **value} for key, value in chunk])
        chunks.append(chunk_df)  # Append the chunk DataFrame

In [ ]:
# Concatenate all chunks into a single DataFrame
cot_df = pd.concat(chunks, ignore_index=True)

In [ ]:
#Get the CoT task in the dataset which has the maximum number of samples
print(cot_df.groupby('task').size().idxmax())
print(cot_df.groupby('task').size().max())

aeslc
10000


In [ ]:
cot_df[cot_df['task'] == 'aeslc']

,id,source,target,config,task,prompt
8,152090,"During this critical time, it is imperative th...",2001 Management Conference,none,aeslc,2
100,150381,What is the subject line for this email?\n\nMi...,Q&As,none,aeslc,0
248,156996,This is an email\nI thought you might like a c...,Sir Mark Moody-Stuart,none,aeslc,6
250,159051,"Write an email with the subject line ""Misc"".",Had a great time this weekend.\nI'm sure glad ...,none,aeslc,9
306,151181,Write a subject line for this message:\n\nI ju...,"Invitation to a Happy ""HOBY"" Hour with Hugh ...",none,aeslc,1
...,...,...,...,...,...,...
447355,151888,Write a subject line for this message:\n\nHi t...,FuelCell offering,none,aeslc,1
447357,157912,"Terri, spoke with Fletch on the SWECI pricing ...",SWECI Pricing,none,aeslc,7
447383,150130,What is the subject line for this email?\n\nBi...,LADWP,none,aeslc,0
447399,155338,This is the content of an email: I will be lea...,Leaving Early...,none,aeslc,5


In [ ]:
#Get samples from the dataset corresponding to the 'aeslc' CoT task and divide them into train and test sets
cot_df_train, cot_df_test = train_test_split(cot_df[cot_df['task'] == 'aeslc'], test_size=0.2)

In [ ]:
#Loading the Flan T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")

#Add prefix to the sentences, tokenize the text, and set the labels
def preprocess_function(examples):
    # The "inputs" are the tokenized source:
    model_inputs = tokenizer([str(i) for i in examples['source']], truncation=True, padding='max_length')
    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=[str(i) for i in examples['target']], truncation=True, padding='max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
#Preprocess train and test data
cot_df_train = preprocess_function(cot_df_train)
cot_df_test = preprocess_function(cot_df_test)

In [ ]:
#Create train and test datasets from the preprocessed data
cot_df_train = TensorDataset(torch.tensor(cot_df_train['input_ids']), torch.tensor(cot_df_train['attention_mask']), torch.tensor(cot_df_train['labels']))
cot_df_test = TensorDataset(torch.tensor(cot_df_test['input_ids']), torch.tensor(cot_df_test['attention_mask']), torch.tensor(cot_df_test['labels']))

In [ ]:
#Create train and test data loaders
train_dataloader = DataLoader(cot_df_train, batch_size=32)
test_dataloader = DataLoader(cot_df_test, batch_size=32)

# Model

In [ ]:
#Loading pre-trained Flan T5 model
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
#Understand model structure
for n, p in model.named_children():
  print(n, p)

shared Embedding(32128, 512)
encoder T5Stack(
  (embed_tokens): Embedding(32128, 512)
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): Linear(in_features=512, out_features=384, bias=False)
            (k): Linear(in_features=512, out_features=384, bias=False)
            (v): Linear(in_features=512, out_features=384, bias=False)
            (o): Linear(in_features=384, out_features=512, bias=False)
            (relative_attention_bias): Embedding(32, 6)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): T5LayerFF(
          (DenseReluDense): T5DenseGatedActDense(
            (wi_0): Linear(in_features=512, out_features=1024, bias=False)
            (wi_1): Linear(in_features=512, out_features=1024, bias=False)
            (wo): Linear(in_features=1024, out_features=512, bias=False)
            (dropout): 

In [ ]:
#Number of trainable parameters in the original model
sum([p.numel() for p in model.parameters() if p.requires_grad is True])

76961152

In [ ]:
# Define a LoRA Layer which has A, B, alpha and rank parameters
class LoRALayer(torch.nn.Module):
  def __init__(self, in_dim, out_dim, rank, alpha):
    super().__init__()
    # Initialize A to be a parameter matrix of dimension in_dim by rank
    self.A = nn.Parameter(torch.empty(in_dim, rank))
    # Initialize all the elements of A via kaiming_uniform with a equal to sqrt(5)
    nn.init.kaiming_uniform_(self.A, a=math.sqrt(5))
    # Initialize B to be a zero parameter matrix of the appropriate dimensions
    self.B = nn.Parameter(torch.zeros(rank, out_dim))
    self.alpha = alpha
    self.rank = rank

  def forward(self, x):
    # Pass x through the LoRA layer and return the new x
    x = (self.alpha/self.rank)*(x @ (self.A@self.B))
    return x

# Define a class LoRALinear which has a linear layer and a LoRA layer on top
class LoRALinear(torch.nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    self.linear = linear
    self.lora = LoRALayer(linear.in_features, linear.out_features, rank, alpha)
    self.weight = None # Each layer in Flan T5 model must have a weight attribute

  def forward(self, x):
    # Pass x through the linear layer and also the lora layer
    return self.linear(x) + self.lora(x)

def lora_linear_replace(model, rank, alpha):
  # Use model.named_children to go through all layers and if the layer is Linear replace that layer with LoRALinear
  for name, module in model.named_children():
    if isinstance(module, nn.Linear):
      setattr(model, name, LoRALinear(module, rank, alpha))
    else:
      # Alterntively, recursively apply the same function to child modules so that other Linear layers get replaced
      lora_linear_replace(module, rank, alpha)

In [ ]:
#Freeze all weights in original model
for param in model.parameters():
  param.requires_grad = False

#Change model structure to include LoRA layers; now only newly added LoRA weights will be trainable
lora_linear_replace(model, 8, 8)

In [ ]:
#Check updated model structure
for n, p in model.named_children():
  print(n, p)

shared Embedding(32128, 512)
encoder T5Stack(
  (embed_tokens): Embedding(32128, 512)
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): LoRALinear(
              (linear): Linear(in_features=512, out_features=384, bias=False)
              (lora): LoRALayer()
            )
            (k): LoRALinear(
              (linear): Linear(in_features=512, out_features=384, bias=False)
              (lora): LoRALayer()
            )
            (v): LoRALinear(
              (linear): Linear(in_features=512, out_features=384, bias=False)
              (lora): LoRALayer()
            )
            (o): LoRALinear(
              (linear): Linear(in_features=384, out_features=512, bias=False)
              (lora): LoRALayer()
            )
            (relative_attention_bias): Embedding(32, 6)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=F

In [ ]:
#Number of trainable parameters in updated model
sum([p.numel() for p in model.parameters() if p.requires_grad is True])

1539072

# Finetuning

In [ ]:
#Putting model on GPU and defining optimizer for fine tuning
model = model.to("cuda")
optimizer = optimizer = torch.optim.AdamW(model.parameters(),lr = 5e-5,eps = 1e-08)

In [ ]:
#Record the total time taken for fine tuning
start_time = time.time()

# Main training loop
for _ in range(2):

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        batch = [element.to('cuda') for element in batch]

        # Unpack the batch
        b_token_ids, b_attn_masks, b_labels = batch

        # Set gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(b_token_ids, attention_mask=b_attn_masks, labels=b_labels)

        # Backward pass
        train_output.loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_token_ids.shape[0]
        nb_tr_steps += 1

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))

end_time = time.time()
print(end_time-start_time)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.



	 - Train loss: 31.8086

	 - Train loss: 8.1404
318.00232553482056
